In [185]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import string
import pickle
import nltk

from nltk.stem import WordNetLemmatizer




# pip install textblob
#nltk.download('words')

In [186]:
data_set_1= pd.read_json('recipes_raw_nosource_ar.json', orient='index')
data_set_2 = pd.read_json('recipes_raw_nosource_epi.json', orient = 'index')
data_set_3 = pd.read_json('recipes_raw_nosource_fn.json', orient='index')

In [187]:
recipe_data = pd.concat([data_set_1, data_set_2, data_set_3])

In [188]:
recipe_data.dropna(inplace=True)

In [189]:
recipe_data = recipe_data.reset_index()
recipe_data.head()

,index,picture_link,title,ingredients,instructions
0,5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,QyrvGdGNMBA2lDdciY0FjKu.77MM0Oe,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ..."
1,clyYQv.CplpwJtjNaFGhx0VilNYqRxu,LVW1DI0vtlCrpAhNSEQysE9i/7rJG56,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar ADVERTISEMENT, 1/2...",Preheat oven to 350 degrees F (175 degrees C)....
2,BmqFAmCrDHiKNwX.IQzb0U/v0mLlxFu,0SO5kdWOV94j6EfAVwMMYRM3yNN8eRi,Best Chocolate Chip Cookies,"[1 cup butter, softened ADVERTISEMENT, 1 cup w...",Preheat oven to 350 degrees F (175 degrees C)....
3,N.jCksRjB4MFwbgPFQU8Kg.yF.XCtOi,YCnbhplMgiraW4rUXcybgSEZinSgljm,Homemade Mac and Cheese Casserole,[8 ounces whole wheat rotini pasta ADVERTISEME...,Preheat oven to 350 degrees F. Line a 2-quart ...
4,kq.naD.8G19M4UU9dVvJgHtpfo.l/eC,jRnWGDXDdyOg3rta4/HVAR2rD19XubC,Banana Banana Bread,"[2 cups all-purpose flour ADVERTISEMENT, 1 tea...",Preheat oven to 350 degrees F (175 degrees C)....


In [190]:
recipe_data.drop('index', axis=1, inplace=True)
recipe_data.head()

,picture_link,title,ingredients,instructions
0,QyrvGdGNMBA2lDdciY0FjKu.77MM0Oe,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ..."
1,LVW1DI0vtlCrpAhNSEQysE9i/7rJG56,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar ADVERTISEMENT, 1/2...",Preheat oven to 350 degrees F (175 degrees C)....
2,0SO5kdWOV94j6EfAVwMMYRM3yNN8eRi,Best Chocolate Chip Cookies,"[1 cup butter, softened ADVERTISEMENT, 1 cup w...",Preheat oven to 350 degrees F (175 degrees C)....
3,YCnbhplMgiraW4rUXcybgSEZinSgljm,Homemade Mac and Cheese Casserole,[8 ounces whole wheat rotini pasta ADVERTISEME...,Preheat oven to 350 degrees F. Line a 2-quart ...
4,jRnWGDXDdyOg3rta4/HVAR2rD19XubC,Banana Banana Bread,"[2 cups all-purpose flour ADVERTISEMENT, 1 tea...",Preheat oven to 350 degrees F (175 degrees C)....


In [191]:
recipe_data.drop('picture_link', axis=1, inplace=True)

In [192]:
recipe_data.head()

,title,ingredients,instructions
0,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ..."
1,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar ADVERTISEMENT, 1/2...",Preheat oven to 350 degrees F (175 degrees C)....
2,Best Chocolate Chip Cookies,"[1 cup butter, softened ADVERTISEMENT, 1 cup w...",Preheat oven to 350 degrees F (175 degrees C)....
3,Homemade Mac and Cheese Casserole,[8 ounces whole wheat rotini pasta ADVERTISEME...,Preheat oven to 350 degrees F. Line a 2-quart ...
4,Banana Banana Bread,"[2 cups all-purpose flour ADVERTISEMENT, 1 tea...",Preheat oven to 350 degrees F (175 degrees C)....


In [193]:
recipe_data['ingredients'][0]

['2 (10.75 ounce) cans condensed cream of mushroom soup ADVERTISEMENT',
 '1 (1 ounce) package dry onion soup mix ADVERTISEMENT',
 '1 1/4 cups water ADVERTISEMENT',
 '5 1/2 pounds pot roast ADVERTISEMENT',
 'ADVERTISEMENT']

## Remove advertisements:

In [194]:
count=0
for i in recipe_data['ingredients']:
    if 'ADVERTISEMENT' in i:
        count+=1

count

39517

In [195]:
recipe_data['ingredients'] = recipe_data['ingredients'].apply(lambda x: [element.replace('ADVERTISEMENT', '') for element in x])

In [196]:
recipe_data.isna().sum()

title           0
ingredients     0
instructions    0
dtype: int64

## Create a column with all the text from the recipe:

In [197]:
recipe_data['full_text'] = ('Recipe title: ' + recipe_data['title']
                            + '. Ingredients: ' + recipe_data['ingredients'].apply(lambda x: ', '.join(x))
                            + '. Instructions: ' + recipe_data['instructions'])

## Remove punctuation:

In [198]:
def remove_punctuation(text):
    cleaned_text = ''.join([i for i in text if i not in string.punctuation])
    
    return cleaned_text

In [199]:
recipe_data['full_text'] = recipe_data['full_text'].apply(lambda x: remove_punctuation(x.lower()))
recipe_data.head()

,title,ingredients,instructions,full_text
0,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ...",recipe title awesome slow cooker pot roast ing...
1,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar , 1/2 cup ketchup ...",Preheat oven to 350 degrees F (175 degrees C)....,recipe title brown sugar meatloaf ingredients ...
2,Best Chocolate Chip Cookies,"[1 cup butter, softened , 1 cup white sugar , ...",Preheat oven to 350 degrees F (175 degrees C)....,recipe title best chocolate chip cookies ingre...
3,Homemade Mac and Cheese Casserole,"[8 ounces whole wheat rotini pasta , 3 cups fr...",Preheat oven to 350 degrees F. Line a 2-quart ...,recipe title homemade mac and cheese casserole...
4,Banana Banana Bread,"[2 cups all-purpose flour , 1 teaspoon baking ...",Preheat oven to 350 degrees F (175 degrees C)....,recipe title banana banana bread ingredients 2...


## Correct misspeled words:

In [200]:
def correct_sentence(sentence):
    sent = TextBlob(sentence)
    result = sent.correct()
    return result

In [201]:
recipe_data['full_text'] = recipe_data['full_text'].apply(lambda x: correct_sentence(x))
recipe_data['full_text'][0]

KeyboardInterrupt: 

## Tokenize:

In [120]:
def tokenization(text):
    tokens = nltk.word_tokenize(text)
    
    return tokens

In [121]:
recipe_data['full_text'] = recipe_data['full_text'].apply(lambda x: tokenization(x))
recipe_data['full_text'][0]

['recipe',
 'title',
 'awesome',
 'slow',
 'cooker',
 'pot',
 'roast',
 'ingredients',
 '2',
 '1075',
 'ounce',
 'cans',
 'condensed',
 'cream',
 'of',
 'mushroom',
 'soup',
 '1',
 '1',
 'ounce',
 'package',
 'dry',
 'onion',
 'soup',
 'mix',
 '1',
 '14',
 'cups',
 'water',
 '5',
 '12',
 'pounds',
 'pot',
 'roast',
 'instructions',
 'in',
 'a',
 'slow',
 'cooker',
 'mix',
 'cream',
 'of',
 'mushroom',
 'soup',
 'dry',
 'onion',
 'soup',
 'mix',
 'and',
 'water',
 'place',
 'pot',
 'roast',
 'in',
 'slow',
 'cooker',
 'and',
 'coat',
 'with',
 'soup',
 'mixture',
 'cook',
 'on',
 'high',
 'setting',
 'for',
 '3',
 'to',
 '4',
 'hours',
 'or',
 'on',
 'low',
 'setting',
 'for',
 '8',
 'to',
 '9',
 'hours']

## Remove the stop words:

In [122]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [123]:
def remove_stopwords(text):
    return [word for word in text if word not in stopwords]

In [124]:
recipe_data['full_text'] = recipe_data['full_text'].apply(lambda x: remove_stopwords(x))
recipe_data['full_text'][0]

['recipe',
 'title',
 'awesome',
 'slow',
 'cooker',
 'pot',
 'roast',
 'ingredients',
 '2',
 '1075',
 'ounce',
 'cans',
 'condensed',
 'cream',
 'mushroom',
 'soup',
 '1',
 '1',
 'ounce',
 'package',
 'dry',
 'onion',
 'soup',
 'mix',
 '1',
 '14',
 'cups',
 'water',
 '5',
 '12',
 'pounds',
 'pot',
 'roast',
 'instructions',
 'slow',
 'cooker',
 'mix',
 'cream',
 'mushroom',
 'soup',
 'dry',
 'onion',
 'soup',
 'mix',
 'water',
 'place',
 'pot',
 'roast',
 'slow',
 'cooker',
 'coat',
 'soup',
 'mixture',
 'cook',
 'high',
 'setting',
 '3',
 '4',
 'hours',
 'low',
 'setting',
 '8',
 '9',
 'hours']

## Lemmatize:

In [125]:
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    return [wordnet_lemmatizer.lemmatize(word) for word in text]

In [126]:
recipe_data['full_text'] = recipe_data['full_text'].apply(lambda x: lemmatize(x))
recipe_data['full_text'][0]

['recipe',
 'title',
 'awesome',
 'slow',
 'cooker',
 'pot',
 'roast',
 'ingredient',
 '2',
 '1075',
 'ounce',
 'can',
 'condensed',
 'cream',
 'mushroom',
 'soup',
 '1',
 '1',
 'ounce',
 'package',
 'dry',
 'onion',
 'soup',
 'mix',
 '1',
 '14',
 'cup',
 'water',
 '5',
 '12',
 'pound',
 'pot',
 'roast',
 'instruction',
 'slow',
 'cooker',
 'mix',
 'cream',
 'mushroom',
 'soup',
 'dry',
 'onion',
 'soup',
 'mix',
 'water',
 'place',
 'pot',
 'roast',
 'slow',
 'cooker',
 'coat',
 'soup',
 'mixture',
 'cook',
 'high',
 'setting',
 '3',
 '4',
 'hour',
 'low',
 'setting',
 '8',
 '9',
 'hour']

## Check for spelling errors 

In [127]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [128]:
# Get the set of English words from the NLTK corpus
english_words = set(nltk.corpus.words.words())

In [129]:
recipe_data['full_text'][0]

['recipe',
 'title',
 'awesome',
 'slow',
 'cooker',
 'pot',
 'roast',
 'ingredient',
 '2',
 '1075',
 'ounce',
 'can',
 'condensed',
 'cream',
 'mushroom',
 'soup',
 '1',
 '1',
 'ounce',
 'package',
 'dry',
 'onion',
 'soup',
 'mix',
 '1',
 '14',
 'cup',
 'water',
 '5',
 '12',
 'pound',
 'pot',
 'roast',
 'instruction',
 'slow',
 'cooker',
 'mix',
 'cream',
 'mushroom',
 'soup',
 'dry',
 'onion',
 'soup',
 'mix',
 'water',
 'place',
 'pot',
 'roast',
 'slow',
 'cooker',
 'coat',
 'soup',
 'mixture',
 'cook',
 'high',
 'setting',
 '3',
 '4',
 'hour',
 'low',
 'setting',
 '8',
 '9',
 'hour']

In [137]:
def check_spelling(text):
    misspelled_words = [word for word in text if word not in english_words]
    misspelled_words = [el for el in misspelled_words if not el.isdigit()]
    return misspelled_words

In [144]:
wrong_words = set()

for row in recipe_data['full_text']:
    words = check_spelling(row)
    wrong_words.update(words)

len(wrong_words)

30661

In [177]:
from textblob import TextBlob

In [182]:
sent = TextBlob('if you need more advanced spelling correction capabilqities, you might wtant to explore dedi1cated spelling corffrection libraries such as')

print(sent.correct())

if you need more advanced spelling correction capabilities, you might want to explore dedicated spelling correction libraries such as
